# Resultados

## Inicializando o DataFrame de Resposta

In [158]:
import pandas as pd

numberOfTopics = 10

answerDF = pd.DataFrame(index=range(numberOfTopics),columns=['TopicID', 'Tags', 'MeanAnswerTime', 'TopicCount'], dType=[])
answerDF[['MeanAnswerTime', 'TopicCount', 'Tags']]= answerDF[['MeanAnswerTime', 'TopicCount', 'Tags']].fillna(0)
answerDF.TopicID = answerDF.index
answerDF

,TopicID,Tags,MeanAnswerTime,TopicCount
0,0,0,0,0
1,1,0,0,0
2,2,0,0,0
3,3,0,0,0
4,4,0,0,0
5,5,0,0,0
6,6,0,0,0
7,7,0,0,0
8,8,0,0,0
9,9,0,0,0


## Inserindo Um Novo Resultado no DataFrame de Resposta

In [157]:
def linkDocumentTopic(line, topicID):
    timeDiff = calculateResponseTime([line['CreationDate'], line['Answer Date']])
    
    answerTopic = answerDF.loc[topicID]
    
    answerDF.loc[topicID, 'MeanAnswerTime'] = (answerTopic.MeanAnswerTime * answerTopic.TopicCount + timeDiff) / (answerTopic.TopicCount + 1)
    answerDF.loc[topicID, 'TopicCount'] += 1
    
    if(answerDF.loc[topicID, 'Tags'] == 0):
        answerDF.at[topicID, 'Tags'] = ",".join(set(line['Tags'][1:-1].split('><')))
    else: 
        answerDF.loc[topicID, 'Tags'] = ",".join(set(answerTopic.Tags.split(',')).union(set(line['Tags'][1:-1].split('><'))))
    

In [155]:
line = {
    'CreationDate': ['2012-06-27 13:51:36'],
    'Answer Date': ['2012-06-27 13:56:42'],
    'Tags': ['<java><c++><performance><cpu-architecture><teste>'],
}

lineDF = pd.DataFrame.from_dict(line)

linkDocumentTopic(lineDF.iloc[0], 0);

{'c++', 'performance', 'cpu-architecture', 'teste', 'java'}


In [169]:
answerDF

,TopicID,Tags,MeanAnswerTime,TopicCount
0,0,0,0,0
1,1,0,0,0
2,2,0,0,0
3,3,0,0,0
4,4,0,0,0
5,5,0,0,0
6,6,0,0,0
7,7,0,0,0
8,8,0,0,0
9,9,0,0,0


## Métricas

### Diferença de Tempo de Resposta

In [162]:
def calculateExpectedTimeDifference(expectedTime, actualTime):
    return expectedTime - actualTime

### Porcentagem de Presença das Tags

In [164]:
def calculatePercentageOfTags(expectedTags, actualTags):
    expectedSet = set(expectedTags.split(','))
    actualSet = set(actualTags.split(','))
    
    return len(expectedSet.intersection(actualSet))/len(actualSet)

## Teste do Modelo

In [ ]:
def TestModel(model, answerDF, testDataFrame):
    timeDifferenceResults = []
    tagsPercentageResults = []
    
    for document in testDataFrame:
        topicID = model.try(document) #TODO MODEL
        timeDifferenceResults.append(calculateExpectedTimeDifference(answerDF.loc[topicID].MeanAnswerTime, calculateResponseTime([document['CreationDate'], document['Answer Date']])))
        tagsPercentageResults.append(calculatePercentageOfTags(answerDF.loc[topicID].Tags, document.Tags))
    
    testDateFrame.assign(timeDifference=timeDifferenceResults)
    testDateFrame.assign(tagsPercentage=tagsPercentageResults)
    
    testDataFrame.describe()